<a href="https://colab.research.google.com/github/wikibook/machine-learning/blob/2.0/jupyter_notebook/5.퍼셉트론_AND_연산.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 퍼셉트론 실습
실습에 앞서, 먼저 텐서플로우를 임포트합니다.

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf

# 항상 같은 결과를 갖기 위해 랜덤 시드 설정
tf.random.set_seed(678)

# 상수 설정
코드의 이해도를 높히기 위해서, 1은 True, 0은 False, 그리고 편형값(bias)은 1로 지정합니다.

In [2]:
T = 1.0
F = 0.0
bias = 1.0

# AND, OR 연산의 이해
실습에 앞서서, AND, OR 연산의 입력값과 그에 따른 출력값을 알아봅시다.
![Image of Perceptron](https://raw.githubusercontent.com/captainchargers/deeplearning/master/img/truth_table.png)

# 데이터 획득
아래 AND, OR, XOR 실습 데이터를 함수를 호출하여 받으실 수 있습니다.

In [3]:
def get_AND_data():
    X = [
    [F, F, bias],
    [F, T, bias],
    [T, F, bias],
    [T, T, bias]
    ]
    
    y = [
        [F],
        [F],
        [F],
        [T]
    ]
    
    return X, y

def get_OR_data():
    X = [
    [F, F, bias],
    [F, T, bias],
    [T, F, bias],
    [T, T, bias]
    ]
    
    y = [
        [F],
        [T],
        [T],
        [T]
    ]
    
    return X, y

def get_XOR_data():
    X = [
    [F, F, bias],
    [F, T, bias],
    [T, F, bias],
    [T, T, bias]
    ]
    
    y = [
        [F],
        [T],
        [T],
        [F]
    ]
    
    return X, y

실습할 연산을 선택합니다.

In [4]:
X, y = get_AND_data()
#X, y = get_OR_data()
#X, y = get_XOR_data()

# 모델 생성
Perceptron 클래스를 아래와 같이 생성합니다.

In [5]:
class Perceptron:
    def __init__(self):
        # 논리연산을 위한 입력값 X, Y와 편향값 (b)를 받을 것이므로, weight를 [3,1]로 설정합니다.  
        # 3은 세개의 입력을 의미하고, 1은 한개의 뉴론임을 의미합니다.
        self.W = tf.Variable(tf.random.normal([3, 1]))
    
    def train(self,X):
        err = 1
        epoch, max_epochs = 0, 20
        while err > 0.0 and epoch < max_epochs:
            epoch += 1
            self.optimize(X)
            # MSE (평균제곱오차)를 관찰하며, 학습이 진행되는 동안, 에러(MSE)가 줄어듬을 확인합니다.
            err = self.mse(y, self.pred(X)).numpy()
            print('epoch:', epoch, 'mse:', err)
    
    @tf.function
    def faster_pred(self, X):
        return self.step(tf.matmul(X, self.W))
    
    def pred(self, X):
        return self.step(tf.matmul(X, self.W))
       
    def mse(self, y, y_hat):
        return tf.reduce_mean(tf.square(tf.subtract(y, y_hat)))
    
    def step(self,x):
        # step(x) = { 1 if x > 0; 0 otherwise }
        return tf.dtypes.cast(tf.math.greater(x, 0), tf.float32)

    def optimize(self, X):
        """
        퍼셉트론은 경사하강법을 사용한 최적화가 불가능합니다.
        매번 학습을 진행할 때마다 가중치를 아래의 룰에 맞게 업데이트합니다.  

        if target == 1 and activation == 0:  
          w_new = w_old + input  

        if target == 0 and activation == 1:  
          w_new = w_old - input  

        위의 두가지 조건은 아래의 코드로 간단히 구현 가능합니다.  
        """
        delta = tf.matmul(X, tf.subtract(y, self.step(tf.matmul(X, self.W))), transpose_a=True)
        self.W.assign(self.W+delta)

# 학습
아래의 코드를 실행시켜 학습을 진행합니다.

In [6]:
perceptron = Perceptron()

In [7]:
perceptron.train(X)

epoch: 1 mse: 0.25
epoch: 2 mse: 0.5
epoch: 3 mse: 0.25
epoch: 4 mse: 0.0


# 테스트

In [15]:
print(perceptron.pred(X).numpy())

[[0.]
 [0.]
 [0.]
 [1.]]


# tf.function 속도 비교

In [9]:
%timeit perceptron.pred(X).numpy()

1.52 ms ± 52.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [10]:
%timeit perceptron.faster_pred(X).numpy()

411 µs ± 13 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [13]:
import timeit

In [38]:
pred_timeit = timeit.timeit(lambda: perceptron.pred(X).numpy(), number=100)
faster_pred_timeit = timeit.timeit(lambda: perceptron.faster_pred(X).numpy(), number=100)
time_diff = round(pred_timeit / faster_pred_timeit, 1)

In [39]:
print("faster_pred(@tf.function) is " + str(time_diff) + " times faster than pred!")

faster_pred(@tf.function) is 5.1 times faster than pred!
